In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [9]:
chromedriver_path = '../chromedriver-mac-x64/chromedriver'
chrome_options = Options()
chrome_options.add_argument("--disable-search-engine-choice-screen")
driver = webdriver.Chrome(options=chrome_options)

In [10]:
driver.get("https://olympics.com/fr/paris-2024/medailles")

In [11]:
element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
)
element.click()

In [12]:
data = []

def scroll_and_collect():
    last_scroll_position = 0
    scroll_step = 300

    while True:
        elements = driver.find_elements(By.CSS_SELECTOR, "div[data-index]")

        for element in elements:
            try:
                span_name = element.find_element(By.CLASS_NAME, 'euzfwma5.emotion-srm-uu3d5n')
                row = [span_name.text]

                spans_81g9w1 = element.find_elements(By.CLASS_NAME, 'e1oix8v91.emotion-srm-81g9w1')
                if len(spans_81g9w1) == 3:
                    row.extend([span.text for span in spans_81g9w1])
                else:
                    continue

                span_5nhv3o = element.find_element(By.CLASS_NAME, 'e1oix8v91.emotion-srm-5nhv3o')
                row.append(span_5nhv3o.text)

                data.append(row)
            except:
                continue

        driver.execute_script(f"window.scrollBy(0, {scroll_step});")
        time.sleep(1)
        
        new_scroll_position = driver.execute_script("return window.pageYOffset;")
        if new_scroll_position == last_scroll_position:
            break
        last_scroll_position = new_scroll_position

scroll_and_collect()

tab = []
for row in data:
    tab.append(row)
    
df = pd.DataFrame(data, columns=["Pays", "Or", "Argent", "Bronze", "Total"])
df = df.drop_duplicates(subset='Pays')

driver.quit()

df


,Pays,Or,Argent,Bronze,Total
0,États-Unis d'Amérique,40,44,42,126
1,République populaire de Chine,40,27,24,91
2,Japon,20,12,13,45
3,Australie,18,19,16,53
4,France,16,26,22,64
...,...,...,...,...,...
355,Pérou,0,0,1,1
356,Qatar,0,0,1,1
357,Singapour,0,0,1,1
358,Slovaquie,0,0,1,1


In [13]:
df.to_csv('Olympics.csv', index=False)